In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import os

In [49]:
class MovieReviews(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        self.train = train 
        self.root = root_dir
        self.path = self.train_file if self.train else self.test_file
        self.reviews = pd.read_csv(self.path, sep="\t")
        self.transform = transform
    
    @property
    def test_file(self):
        return os.path.join(self.root, "test.tsv")
    
    @property
    def train_file(self):
        return os.path.join(self.root, "train.tsv")
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        phrase = self.reviews.iloc[idx, 2]
        if self.transform:
            phrase = self.transform(phrase)
        
        if self.train:
            y = self.reviews.iloc[idx, 3]
            return phrase, y
        else:
            return phrase

In [3]:
movie_train = MovieReviews("./movie_reviews/", train=True, transform=None)

In [50]:
movie_test = MovieReviews("./movie_reviews/", train=False, transform=None)

In [5]:
movie_train.path

'./movie_reviews/train.tsv'

In [9]:
movie_train[0]

('A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
 1)

In [6]:
from torchtext.data.utils import ngrams_iterator
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab 
import torchtext
from torchtext.data import get_tokenizer

In [11]:
tokenizer = get_tokenizer('basic_english')

In [12]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(movie_train))

156060lines [00:14, 11023.52lines/s]


In [14]:
vocab

In [15]:
vocab.lookup_indices(['here', 'is', 'an', 'example'])

[201, 12, 20, 1788]

In [16]:
help(vocab)

Help on Vocab in module torchtext.vocab object:

class Vocab(builtins.object)
 |  Vocab(counter, max_size=None, min_freq=1, specials=('<unk>', '<pad>'), vectors=None, unk_init=None, vectors_cache=None, specials_first=True)
 |  
 |  Defines a vocabulary object that will be used to numericalize a field.
 |  
 |  Attributes:
 |      freqs: A collections.Counter object holding the frequencies of tokens
 |          in the data used to build the Vocab.
 |      stoi: A collections.defaultdict instance mapping token strings to
 |          numerical identifiers.
 |      itos: A list of token strings indexed by their numerical identifiers.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getitem__(self, token)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, counter, max_size=None, min_freq=1, specials=('<unk>', '<pad>'), vectors=None, unk_init=None, vectors_cache=None, specials_first=True)
 |      Create a Vocab object from a collections.Coun

In [38]:
vocab["here"]

201

In [30]:
text_pipeline = lambda x: vocab.lookup_indices(tokenizer(x))
label_pipeline = lambda x: int(x)

In [33]:
label_pipeline(0)

0

In [31]:
text_pipeline('here is the an example')

[201, 12, 2, 20, 1788]

In [17]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64).to(device)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64).to(device)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0).to(device)
    text_list = torch.cat(text_list).to(device)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [18]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [19]:
sentiment_map = {
    0: "negative",
    1: "somewhat negative",
    2: "neutral",
    3: "somewhat positive",
    4: "positive"
}

In [20]:
num_classes = len(sentiment_map.keys())
vocab_size = len(vocab)
embed_size = 64
model = TextClassificationModel(vocab_size, embed_size, num_classes).to(device)

In [21]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
#     with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [22]:
from torch.utils.data.dataset import random_split
#from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

In [25]:
num_train = int(len(movie_train) * 0.95)
split_train_, split_valid_ = \
    random_split(movie_train, [num_train, len(movie_train) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

In [ ]:
test_loader = 

In [35]:
def to_device(data, device=None):
    """Move tensor(s) to chosen device"""
    if not device:
        device = _device

    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """
    Wrap a dataloader to move data to a device
    """
    def __init__(self, dl, device=None):
        self.dl = dl
        if device:
            device = device
        else:
            device = _device

        self.device = device

    def __iter__(self):
        """
        Yield a batch of data after moving it to device
        """
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """
        Number of batches
        """
        return len(self.dl)

In [27]:
train_dataloader = DeviceDataLoader(train_dataloader, device=device)
valid_dataloader = DeviceDataLoader(valid_dataloader, device=device)

In [28]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

In [36]:
import time 
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 2317 batches | accuracy    0.497
| epoch   1 |  1000/ 2317 batches | accuracy    0.501
| epoch   1 |  1500/ 2317 batches | accuracy    0.510
| epoch   1 |  2000/ 2317 batches | accuracy    0.526
-----------------------------------------------------------
| end of epoch   1 | time: 26.13s | valid accuracy    0.532 
-----------------------------------------------------------
| epoch   2 |   500/ 2317 batches | accuracy    0.545
| epoch   2 |  1000/ 2317 batches | accuracy    0.549
| epoch   2 |  1500/ 2317 batches | accuracy    0.559
| epoch   2 |  2000/ 2317 batches | accuracy    0.559
-----------------------------------------------------------
| end of epoch   2 | time: 25.83s | valid accuracy    0.554 
-----------------------------------------------------------
| epoch   3 |   500/ 2317 batches | accuracy    0.584
| epoch   3 |  1000/ 2317 batches | accuracy    0.585
| epoch   3 |  1500/ 2317 batches | accuracy    0.583
| epoch   3 |  2000/ 2317 batches | accuracy